#part1
1-

In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

In [2]:
image_avant=cv2.imread("bob.PNG",cv2.IMREAD_GRAYSCALE)

image_avant[:]=image_avant[:]/2
image_avant

array([[12, 12, 12, ..., 15, 15, 15],
       [12, 12, 12, ..., 15, 15, 15],
       [12, 12, 12, ..., 15, 15, 15],
       ...,
       [46, 46, 46, ..., 25, 25, 25],
       [46, 45, 46, ..., 25, 25, 25],
       [46, 46, 45, ..., 25, 25, 25]], dtype=uint8)

In [3]:
import numpy as np

def filterMiy(img, vois=20):
    h, w = img.shape
    imgmoy = np.zeros_like(img)

    y = 0
    while y < h:
        x = 0
        while x < w:
            if (y < vois/2 or y > h-vois/2 or x < vois/2 or x > w-vois/2):
                imgmoy[y, x] = img[y, x]
            else:
                imgv = img[y-int(vois/2):y+int(vois/2)+1, x-int(vois/2):x+int(vois/2)+1]
                moy = np.mean(imgv)
                imgmoy[y, x] = moy

            x += 1
        y += 1

    return imgmoy

def filterMed(img, vois=20):
    h, w = img.shape
    imgmed = np.zeros_like(img)

    y = 0
    while y < h:
        x = 0
        while x < w:
            if (y < vois/2 or y > h-vois/2 or x < vois/2 or x > w-vois/2):
                imgmed[y, x] = img[y, x]
            else:
                imgv = img[y-int(vois/2):y+int(vois/2)+1, x-int(vois/2):x+int(vois/2)+1]
                t = np.sort(imgv.flatten())
                imgmed[y, x] = t[int((vois*vois/2)+1)]

            x += 1
        y += 1

    return imgmed


In [39]:
import cv2
import numpy as np

def convolve(image, kernel):
    height, width = image.shape
    k_height, k_width = kernel.shape
    pad_height = k_height // 2
    pad_width = k_width // 2

    # Create an empty output image with the same data type as the input image
    output = np.zeros_like(image, dtype=np.float32)

    # Pad the input image
    padded_image = cv2.copyMakeBorder(image, pad_height, pad_height, pad_width, pad_width, cv2.BORDER_CONSTANT)

    # Perform convolution without using range, sum, or np.sum
    i = 0
    while i < height:
        j = 0
        while j < width:
            total_sum = 0
            m = 0
            while m < k_height:
                n = 0
                while n < k_width:
                    total_sum += padded_image[i+m, j+n] * kernel[m, n]
                    n += 1
                m += 1
            output[i, j] = total_sum
            j += 1
        i += 1

    return output


In [7]:
gaussian = np.array([[1, 2, 1], [2, 4, 2], [1, 2, 1]]) / 16
laplacian = np.array([[0, 1, 0], [1, -4, 1], [0, 1, 0]])

image_gaussian=convolve(image_avant,gaussian)
image_laplacian=convolve(image_gaussian,laplacian)

In [5]:
def threshold_filter(image, threshold=128):
    height, width= image.shape

    # Create an empty output image with the same width and height
    output = np.zeros((height, width), dtype=np.uint8)

    i = 0
    while i < height:
        j = 0
        while j < width:
            # Extract intensity (grayscale value) from the pixel
            intensity = image[i, j]

            # Apply thresholding
            if intensity > threshold:
                output[i, j] = 255  # Set to white
            else:
                output[i, j] = 0    # Set to black

            j += 1
        i += 1

    return output

In [9]:
image_threshold=threshold_filter(image_avant,128)

In [3]:
def grayscale_filter(image):
    height, width, channels = image.shape

    # Create an empty output image with the same width and height
    output = np.zeros((height, width), dtype=np.uint8)

    i = 0
    while i < height:
        j = 0
        while j < width:
            # Extract RGB values
            r, g, b = image[i, j]

            # Calculate grayscale value using a simple weighted sum
            gray_value = int(0.299 * r + 0.587 * g + 0.114 * b)

            # Set the grayscale value in the output image
            output[i, j] = gray_value

            j += 1
        i += 1

    return output

In [4]:
image_avant_c=cv2.imread("bob.PNG")
image_grayscale_filter=grayscale_filter(image_avant_c)


In [11]:
def apply_erosion_filter(image, matrix_size):
    filter_offset = (matrix_size - 1) // 2
    morph_reset_value = 255

    new_image = np.zeros_like(image)

    for y in range(filter_offset, image.shape[0] - filter_offset):
        for x in range(filter_offset, image.shape[1] - filter_offset):
            roi = image[y - filter_offset:y + filter_offset + 1, x - filter_offset:x + filter_offset + 1]

            blue = np.min(roi[:, :, 0])
            green = np.min(roi[:, :, 1])
            red = np.min(roi[:, :, 2])

            new_image[y, x] = [blue, green, red]

    return new_image
# Create a structuring element
eroded_image=apply_erosion_filter(image_avant_c,5)
cv2.imshow('Original image', image_avant_c)
cv2.imshow('Eroded image', eroded_image)

# Wait for a key press to close the windows
cv2.waitKey(0)
cv2.destroyAllWindows()

In [12]:
def apply_dilation_filter(image, matrix_size):
    filter_offset = (matrix_size - 1) // 2
    morph_reset_value = 0

    new_image = np.zeros_like(image)

    for y in range(filter_offset, image.shape[0] - filter_offset):
        for x in range(filter_offset, image.shape[1] - filter_offset):
            roi = image[y - filter_offset:y + filter_offset + 1, x - filter_offset:x + filter_offset + 1]

            blue = np.max(roi[:, :, 0])
            green = np.max(roi[:, :, 1])
            red = np.max(roi[:, :, 2])

            new_image[y, x] = [blue, green, red]

    return new_image
dilation_image=apply_dilation_filter(image_avant_c,5)
cv2.imshow('Original image', image_avant_c)
cv2.imshow('Eroded image', dilationfilter)

# Wait for a key press to close the windows
cv2.waitKey(0)
cv2.destroyAllWindows()

In [14]:
def apply_opening_filter(image, matrix_size):
    # Erosion
    eroded_image = apply_erosion_filter(image, matrix_size)
    
    # Dilation
    opened_image = apply_dilation_filter(eroded_image, matrix_size)
    
    return opened_image

opening_image=apply_opening_filter(image_avant_c,5)
cv2.imshow('Original image', image_avant_c)
cv2.imshow('Eroded image', opening_image)

# Wait for a key press to close the windows
cv2.waitKey(0)
cv2.destroyAllWindows()

In [15]:
def apply_closing_filter(image, matrix_size):
    # Dilation
    dilated_image = apply_dilation_filter(image, matrix_size)
    
    # Erosion
    closed_image = apply_erosion_filter(dilated_image, matrix_size)

    return closed_image

closing_image=apply_closing_filter(image_avant_c,5)
cv2.imshow('Original image', image_avant_c)
cv2.imshow('Eroded image', closing_image)

# Wait for a key press to close the windows
cv2.waitKey(0)
cv2.destroyAllWindows()

In [16]:
def apply_morphological_gradient(image, matrix_size):
    # Dilation
    dilated_image = apply_dilation_filter(image, matrix_size)
    # Erosion
    eroded_image = apply_erosion_filter(image, matrix_size)
    # Morphological gradient (difference between dilation and erosion)
    gradient_image = dilated_image - eroded_image
    
    return gradient_image
gradient_image=apply_morphological_gradient(image_avant_c,5)
cv2.imshow('Original image', image_avant_c)
cv2.imshow('Eroded image', gradient_image)

# Wait for a key press to close the windows
cv2.waitKey(0)
cv2.destroyAllWindows()

In [18]:
def apply_top_hat_transform(image, matrix_size):
    # Structuring element for morphological operations
    kernel = np.ones((matrix_size, matrix_size), dtype=np.uint8)

    # Perform opening operation
    opened_image = apply_opening_filter(image, matrix_size)
    # Top-hat transform (difference between the original image and its opening)
    top_hat_image = image-opened_image

    return top_hat_image

top_hat_image=apply_top_hat_transform(image_avant_c,5)
cv2.imshow('Original image', image_avant_c)
cv2.imshow('Eroded image', top_hat_image)

# Wait for a key press to close the windows
cv2.waitKey(0)
cv2.destroyAllWindows()

In [20]:
def apply_black_hat_transform(image, matrix_size):

    closed_image = apply_closing_filter(image, matrix_size)

    # Black-hat transform (difference between the closing of the original image and the original image)
    black_hat_image = closed_image-image

    return black_hat_image
black_hat_image=apply_black_hat_transform(image_avant_c,5)
cv2.imshow('Original image', image_avant_c)
cv2.imshow('Eroded image', black_hat_image)

# Wait for a key press to close the windows
cv2.waitKey(0)
cv2.destroyAllWindows()

In [23]:
def apply_mosaic(image, factor):
    # Get image dimensions
    height, width, _ = image.shape

    # Create an empty image for the mosaic effect
    mosaic_image = np.zeros_like(image)

    # Iterate over mosaic blocks and set each block to the average color of its pixels
    for i in range(0, height, factor):
        for j in range(0, width, factor):
            block = image[i:i+factor, j:j+factor]
            average_color = np.mean(block, axis=(0, 1), dtype=np.uint8)
            mosaic_image[i:i+factor, j:j+factor] = average_color

    return mosaic_image

mosaic_image=apply_mosaic(image_avant_c,1)
cv2.imshow('Original image', image_avant_c)
cv2.imshow('Eroded image', mosaic_image)

# Wait for a key press to close the windows
cv2.waitKey(0)
cv2.destroyAllWindows()


In [51]:
def apply_motion_blur(image, kernel_size):
    # Create a motion blur kernel
    kernel = np.zeros((kernel_size, kernel_size))
    kernel[:, int((kernel_size-1)/2)] = np.ones(kernel_size) / kernel_size

    # Apply the motion blur filter without using .convolve
    motion_blur_image = np.zeros_like(image, dtype=np.float32)
    for i in range(image.shape[2]):  # Iterate over color channels
        motion_blur_image[:, :, i] = np.convolve(image[:, :, i].flatten(), kernel[:, int((kernel_size-1)/2)], mode='same').reshape(image.shape[:2])

    return motion_blur_image.astype(np.uint8)
motion_blur_image=apply_motion_blur(image_avant_c,5)
cv2.imshow('Original image', image_avant_c)
cv2.imshow('Eroded image', motion_blur_image)

# Wait for a key press to close the windows
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
def object_color_detection(image, lower_color, upper_color):
    # Convertir l'image de BGR à HSV (Espace de couleur Hue, Saturation, Value)
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Créer un masque en utilisant la plage de couleurs spécifiée
    color_mask = cv2.inRange(hsv_image, lower_color, upper_color)

    # Appliquer le masque à l'image d'origine pour ne conserver que les pixels de l'objet
    result_image = cv2.bitwise_and(image, image, mask=color_mask)

    return result_image

In [ ]:
lower_green = np.array([40, 40, 40])
upper_green = np.array([80, 255, 255])
result = object_color_detection(image_avant_c, lower_green, upper_green)
cv2.imshow('Original Image', image_avant_c)
cv2.imshow('Color Detection Result', result)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [11]:
def Object_Color_Detection(image, color_range):
    # Convert the image to the HSV color space
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Define the lower and upper bounds of the color range
    lower_bound = np.array(color_range[0])
    upper_bound = np.array(color_range[1])

    # Create a mask for the desired color
    mask = cv2.inRange(hsv_image, lower_bound, upper_bound)

    # Fill holes in the mask
    filled_mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, np.ones((5, 5), np.uint8))

    # Find the average color of the object
    average_color = cv2.mean(image, filled_mask)[:3].astype(np.uint8)

    # Find the bounding box of the object
    bounding_box = None
    for i in range(0, mask.shape[0]):
        for j in range(0, mask.shape[1]):
            if mask[i, j] == 255:
                if bounding_box is None:
                    bounding_box = [i, j, i, j]
                else:
                    bounding_box[0] = min(bounding_box[0], i)
                    bounding_box[1] = min(bounding_box[1], j)
                    bounding_box[2] = max(bounding_box[2], i)
                    bounding_box[3] = max(bounding_box[3], j)

    # Draw the bounding box on the image
    if bounding_box is not None:
        cv2.rectangle(image, (bounding_box[1], bounding_box[0]), (bounding_box[3], bounding_box[2]), (0, 255, 0), 3)

    return image, average_color


def detect_inrange(image, low, high, min_area, max_area):
    points = []

    image = cv2.blur(image, (5, 5))
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv_image, low, high)

    contours, _ = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key=lambda x: cv2.contourArea(x), reverse=True)

    for contour in contours:
        area = cv2.contourArea(contour)
        if min_area < area < max_area:
            (x, y), radius = cv2.minEnclosingCircle(contour)
            points.append((int(x), int(y), int(radius), int(area)))

    return image, mask, points

In [12]:
color_range = [(0, 100, 100), (100, 255, 255)]
detected_image = Object_Color_Detection(image_avant_c, color_range)
cv2.imshow('Detected Image', detected_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

AttributeError: 'tuple' object has no attribute 'astype'